# Grundlæggende regression: Forudsig brændstofeffektivitet

I et *regressions*-problem er målet at forudsige outputtet af en kontinuerlig værdi, som for eksempel en pris eller en sandsynlighed. Dette står i kontrast til et *klassifikations*-problem, hvor målet er at vælge en klasse fra en liste af klasser (for eksempel når et billede indeholder et æble eller en appelsin, at genkende hvilken frugt der er på billedet).

Denne vejledning bruger det klassiske [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg) datasæt og demonstrerer, hvordan man bygger modeller til at forudsige brændstofeffektiviteten for biler fra slutningen af 1970'erne og begyndelsen af 1980'erne. For at gøre dette vil du forsyne modellerne med en beskrivelse af mange biler fra denne tidsperiode. Denne beskrivelse inkluderer attributter som cylindere, slagvolumen, hestekræfter og vægt.

Dette eksempel bruger Keras API'et. (I kan bruge Keras [vejledninger](https://www.tensorflow.org/tutorials/keras) og [guides](https://www.tensorflow.org/guide/keras) for at lære mere.)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Laver NumPy-printouts lettere at læse.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

## "The Auto MPG" datasæt

# Datasættet er tilgængeligt fra [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/).


### Hent dataene
Først download og importer datasættet ved hjælp af pandas:

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [ ]:
dataset = raw_dataset.copy()
dataset.tail()

### Rens dataene

Datasættet indeholder et par ukendte værdier:

In [ ]:
dataset.isna().sum()

Fjern rækker med ukendte værdier:


In [ ]:
dataset = dataset.dropna()

Kolonnen `"Origin"` er kategorisk, ikke numerisk. Så det næste skridt er at one-hot encode værdierne i kolonnen med [pd.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html).

Man kan læse mere om one-hot encoding her: [ml-one-hot-encoding](https://www.geeksforgeeks.org/ml-one-hot-encoding/)


In [ ]:
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

In [ ]:
dataset = pd.get_dummies(dataset, columns=['Origin'], prefix='', prefix_sep='')
dataset.tail()

### Del dataene op i trænings- og test-sæt

Nu skal datasættet opdeles i et træningssæt og et test-sæt. Du vil bruge test-sættet i den endelige evaluering af dine modeller.

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

### Inspicer dataene

Gennemgå den fælles fordeling af et par kolonnepar fra træningssættet.

Den øverste række antyder, at brændstofeffektiviteten (MPG) er en funktion af alle de andre parametre. De andre rækker indikerer, at de er funktioner af hinanden.

In [ ]:
sns.pairplot(train_dataset[['MPG', 'Cylinders', 'Displacement', 'Weight']], diag_kind='kde')

Lad os også kontrollere de overordnede statistikker. Bemærk, hvordan hver funktion dækker et meget anderledes område:

In [ ]:
train_dataset.describe().transpose()

### Opdel funktioner fra mærker

Separér målvariablen—mærket—fra funktionerne. Denne mærke er den værdi, du vil træne modellen til at forudsige.


In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

## Normalization

I tabellen med statistikker er det let at se, hvor meget forskellige funktionerne er: 


In [ ]:
train_dataset.describe().transpose()[['mean', 'std']]

Det er en god praksis at normalisere funktioner, der bruger forskellige skalaer og områder. 

En grund til, at dette er vigtigt, er, at funktionerne ganges med modellens vægte. Så påvirkes skalaen af output og gradienten af skalaen af input.

Selvom en model *muligvis* kan konvergere uden funktionsnormalisering, gør normalisering træning meget mere stabil. 


### Normalisering lag

`tf.keras.layers.Normalization` er en ren og simpel måde at tilføje funktionsnormalisering til vores model. 

Første trin er at oprette laget:


In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)

Derefter passer vi tilstanden af forbehandlingslaget til data ved at kalde `Normalization.adapt`:

In [ ]:
normalizer.adapt(np.array(train_features))


Vi beregner gennemsnittet og variansen, og gemmer dem i laget:

In [ ]:
print(normalizer.mean.numpy())

Når laget kaldes, returnerer det inputdataen, med hver funktion normaliseret uafhængigt:

In [ ]:
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

## Linear regression | Lineær regression

Før vi bygger en neural network model, starter vi med lineær regression ved hjælp af en og flere variable.

### Linear regression med én variabel

Begynd med en enkeltvariabel lineær regression for at forudsige `'MPG'` fra `'Horsepower'`.

At træne en model med `tf.keras` starter typisk med at definere modelarkitekturen. Brug en `tf.keras.Sequential` model, som [repræsenterer en sekvens af trin](https://www.tensorflow.org/guide/keras/sequential_model).

Der er to trin i din enkeltvariabel lineære regressionsmodel:

- Normaliser inputfunktionerne `'Horsepower'` ved hjælp af `tf.keras.layers.Normalization` preprocessing-laget.
- Anvend en lineær transformation ($y = mx+b$) for at producere 1 output ved hjælp af et lineært lag (`tf.keras.layers.Dense`).

Antallet af _inputs_ kan enten sættes ved `input_shape` argumentet eller automatisk, når modellen køres første gang.

Først, lav en NumPy array, der består af funktionen `'Horsepower'`. Så opretter vi `tf.keras.layers.Normalization` laget og passer tilstanden af laget til data:


In [ ]:
horsepower = np.array(train_features['Horsepower'])

horsepower_normalizer = layers.Normalization(input_shape=[1,], axis=None)
horsepower_normalizer.adapt(horsepower)

Byg Keras Sequential model:

In [ ]:
horsepower_model = tf.keras.Sequential([
    horsepower_normalizer,
    layers.Dense(units=1)
])

horsepower_model.summary()

Denne model vil forudsige `'MPG'` fra `'Horsepower'`.

Kør den ubegyndelige model på de første 10 'Horsepower' værdier. Outputtet vil ikke være godt, men bemærk, at det har den forventede form af `(10, 1)`:



In [ ]:
horsepower_model.predict(horsepower[:10])

Når vi bygger modellen, konfigurerer vi træningen ved hjælp af `tf.keras.Model.compile` metode. De vigtigste argumenter at konfigurere er `loss` og `optimizer`, da disse definerer, hvad der vil blive optimalt (`mean_absolute_error`) og hvordan (ved hjælp af `tf.keras.optimizers.Adam`).


In [ ]:
horsepower_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

Brug Keras `Model.fit` til at udføre træningen for 100 epoker:


In [ ]:
%%time
history = horsepower_model.fit(
    train_features['Horsepower'],
    train_labels,
    epochs=100,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

Visualiser modellens træningsprogres ved hjælp af stats gemt i `history` objektet:


In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

In [ ]:
plot_loss(history)

Indsamle resultaterne på test-sættet for senere:

In [ ]:
test_results = {}

test_results['horsepower_model'] = horsepower_model.evaluate(
    test_features['Horsepower'],
    test_labels, verbose=0)

Siden dette er en enkeltvariabel regression, er det let at se modellens forudsagelser som en funktion af input:

In [ ]:
x = tf.linspace(0.0, 250, 251)
y = horsepower_model.predict(x)

In [ ]:
def plot_horsepower(x, y):
  plt.scatter(train_features['Horsepower'], train_labels, label='Data')
  plt.plot(x, y, color='k', label='Predictions')
  plt.xlabel('Horsepower')
  plt.ylabel('MPG')
  plt.legend()

In [ ]:
plot_horsepower(x, y)

### Lineær regression med flere inputs

Du kan bruge en næsten identisk opsætning til at forudsige baseret på flere inputs. Denne model gør det samme $y = mx+b$, men $m$ er en matrix og $b$ er en vektor. 

Opret en to-trin Keras Sequential model igen med det første lag som `normalizer` (`tf.keras.layers.Normalization(axis=-1)`) du definerede tidligere og tilpasset hele datasættet:


In [ ]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

Når du kalder `Model.predict` på en batch af inputs, producerer det `units=1` outputs for hver eksempel:

In [ ]:
linear_model.predict(train_features[:10])

Når du kalder modellen, vil dens vægtmatricer blive bygget—sørg for, at `kernel` vægte (den $m$ i $y=mx+b$) har en form af `(9, 1)`: 

In [ ]:
linear_model.layers[1].kernel

Konfigurer modellen med Keras `Model.compile` og træn med `Model.fit` for 100 epoker: 

In [ ]:
linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
%%time
history = linear_model.fit(
    train_features,
    train_labels,
    epochs=100,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

Denne model opnår en meget lavere trænings- og valideringsfejl end `horsepower_model`, som havde en input:

In [ ]:
plot_loss(history)

Indsamle resultaterne på test-sættet for senere:

In [ ]:
test_results['linear_model'] = linear_model.evaluate(
    test_features, test_labels, verbose=0)

## Regression med en Deep Neural Network (DNN) 

I den forrige sektion, implementerede du to lineære modeller for enkelt- og fler-input. 

Her implementerer du enkelt- og fler-input DNN modeller.

Koden er næsten den samme, undtagen at modellen udvides til at inkludere nogle "skjulte" ikke-lineære lag. Navnet "skjult" her betyder, at det ikke er direkte forbundet med input eller output.

Disse modeller vil indeholde et par flere lag end lineære modeller: 

* Normaliseringslaget, som før (med `horsepower_normalizer` for enkelt-input model og `normalizer` for fler-input model). 
* To skjulte, ikke-lineære, `Dense` lag med ReLU (`relu`) aktiveringsfunktionen.
* En lineær `Dense` outputlag.

Begge modeller vil bruge den samme træningsprocedure, så `compile` metode er inkluderet i `build_and_compile_model` funktionen nedenfor.


In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

### Regression med en DNN og et enkelt input

Opret en DNN model med kun `'Horsepower'` som input og `horsepower_normalizer` (defineret tidligere) som normaliseringslag:

In [ ]:
dnn_horsepower_model = build_and_compile_model(horsepower_normalizer)

Denne model har et par flere trænbare parametre end lineære modeller:

In [ ]:
dnn_horsepower_model.summary()

Træn modellen med Keras `Model.fit`:

In [ ]:
%%time
history = dnn_horsepower_model.fit(
    train_features['Horsepower'],
    train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

Denne model gør lidt bedre end lineære enkelt-input `horsepower_model`:

In [ ]:
plot_loss(history)

Hvis du plotter forudsagelserne som en funktion af `'Horsepower'`, bør du bemærke, hvordan denne model udnytter den ikke-lineære natur af skjulte lag: 

In [ ]:
x = tf.linspace(0.0, 250, 251)
y = dnn_horsepower_model.predict(x)

In [ ]:
plot_horsepower(x, y)

Indsamle resultaterne på test-sættet for senere:

In [ ]:
test_results['dnn_horsepower_model'] = dnn_horsepower_model.evaluate(
    test_features['Horsepower'], test_labels,
    verbose=0)

### Regression med en DNN og flere inputs

Gentag den tidligere proces ved hjælp af alle input. Modellens ydeevne forbedres lidt på valideringsdatasættet.

In [ ]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

In [ ]:
%%time
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

In [ ]:
plot_loss(history)

Indsamle resultaterne på test-sættet:

In [ ]:
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

## Ydeevne

Da alle modeller er trænet, kan du gennemgå deres ydeevne på test-sættet:

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T

Disse resultater matcher den valideringsfejl, der blev observeret under træning. 

### Lav forudsagelser

Du kan nu lave forudsagelser med `dnn_model` på test-sættet ved hjælp af Keras `Model.predict` og gennemgå tabet:

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)


Det ser ud til, at modellen forudsiger rimeligt godt.

Nu, kontroller fejldistributionen:

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [MPG]')
_ = plt.ylabel('Count')

Hvis du er tilfreds med modellen, gem den til senere brug med `Model.save`: 

In [ ]:
dnn_model.save('dnn_model')

Hvis du genindlæser modellen, giver den identiske output: 

In [ ]:
reloaded = tf.keras.models.load_model('dnn_model')

test_results['reloaded'] = reloaded.evaluate(
    test_features, test_labels, verbose=0)

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T

## Konklusion

Denne vejledning introducerede et par teknikker til at håndtere et regressionsproblem. Her er et par flere tips, der kan hjælpe: Specielt til jeres projekt i uge nummer 2!


- Mean squared error (MSE) (`tf.keras.losses.MeanSquaredError`) og mean absolute error (MAE) (`tf.keras.losses.MeanAbsoluteError`) er almindelige loss funktioner, der bruges til regressionsproblemer. MAE er mindre følsom over for outliers. Forskellige loss funktioner bruges til klassifikationsproblemer. 
- Ligeledes bruges evalueringsmetrikkerne for regressionsproblemer fra klassifikationsproblemer.
- Når numeriske input data features har værdier med forskellige ranges, bør hver feature skaleres uafhængigt til den samme range.
- Overfitting er et almindeligt problem for DNN modeller, selvom det ikke var et problem i denne vejledning. Besøg [Overfit og underfit](../4.Fit/) vejledningen for yderligere hjælp.